In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap, HeatMapWithTime
import plotly.express as px

In [ ]:
def date_to_time(date):
    day_to_h = {'Lun':0,'Mar':24,'Mer':2*24,'Jeu':3*24,'Ven':4*24,'Sam':5*24,'Dim':6*24}
    day = date[:3]
    hour = int(date[-2:])
    return day_to_h[day] + hour

def time_to_day(time):
    return time[:3]

In [ ]:
velib = pd.read_csv('velibLoading.csv', sep=' ')
velibAdds = pd.read_csv('velibAdds.csv', sep=' ')
velib = pd.concat([velibAdds, velib], axis=1)
#velib = velib.rename(columns={'longitude':'lon', 'latitude':'lat', 'bonus':'hill'})
velib.head()

In [ ]:
days = ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')
velib_long = velib.melt(id_vars=['longitude','latitude','bonus','names'], var_name='time', value_name='load')#.groupby('name').mean()
velib_long['day'] = velib_long['time'].apply(lambda t: t[:3])
velib_long['hour'] = velib_long['time'].apply(lambda t: int(t[-2:]))
velib_long.drop('time', inplace=True, axis=1)
#velib_long['time'] = velib_long['time'].apply(date_to_time)
velib_long.head(10)
#data = []
#for _, d in velib_long.groupby('time'):
#   data.append([[row['lat'], row['long'], row['load']] for _, row in d.iterrows()])

In [ ]:
init_lat = velib_long['lat'].median() #(velib_long['lat'].max() + velib_long['lat'].min()) / 2
init_long = velib_long['long'].median() #(velib_long['long'].max() + velib_long['long'].min()) / 2

m = folium.Map(location=[init_lat, init_long], zoom_start=12)
hm = HeatMapWithTime(data, radius=15)
hm.add_to(m)
m

In [ ]:
load_by_hour = velib_long[['day', 'hour', 'load']].groupby(['day', 'hour']).mean()
hour = np.arange(24)
day_colors = ['#D9F0A3', '#ADDD8E', '#78C679', '#31A354', '#006837', '#6BAED6', '#08519C']
days = ['Lun', 'Mar', 'Mer', 'Jeu', 'Ven', 'Sam', 'Dim']
plt.figure(figsize=(10,6))
for k in range(7):
    plt.plot(hour, load_by_hour.loc[days[k]].values, color=day_colors[k])
plt.show()

In [ ]:
#load_by_hour.reset_index(level=['day', 'hour'], inplace=True)
px.line(load_by_hour, x='hour', y='load', color='day')